In [1]:
    import fiona
    import os,os.path
    from shapely.geometry import shape,mapping
    from shapely.ops import transform
    from functools import partial
    import pyproj
    from fiona.crs import from_epsg
    import osmnx as ox
    import matplotlib.pyplot as plt
    import geopandas as gpd
    import glob
    import json
    import numpy as np
    import requests as rq
    import collections
    import logging
    import time
    from PIL import Image
    from io import BytesIO
    import pandas as pd
    import dateutil.parser
    %matplotlib inline

In [2]:
# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
f_handler = logging.FileHandler('Mapillary_download.log')
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(f_handler)

In [3]:
city = '../Data/GreaterLondon/pointsJson/'
files= glob.glob(city + 'Mapillary*.json')

In [4]:
mapillary_Image_url = 'https://images.mapillary.com/' #<image_key>/thumb-640.jpg

In [5]:
# Final_images = {} 
# for f in files:
#     ImageDict = json.load(open(f,'rb'))
#     for image in ImageDict:
#         if image['properties']['key'] not in Final_images:
#             Final_images[image['properties']['key']] = image

In [8]:
# json.dump(Final_images,open(city+"All_London_images_mapillary.json",'wb'))
Final_images = json.load(open('../Data/GreaterLondon/pointsJson/All_London_images_mapillary.json'))

In [9]:
def FilterByTime(imageDict , ThreshYear, ThreshMonth ):
    filtered = {}
    for k in imageDict:
        yourdate = dateutil.parser.parse(imageDict[k]['properties']['captured_at'])
        if yourdate.year > ThreshYear and yourdate.month > ThreshMonth:
            filtered[k] = imageDict[k]
    return filtered

def getYearDict(imageDict):
    yearFreq = {}
    for k in imageDict:
        yourdate = dateutil.parser.parse(imageDict[k]['properties']['captured_at'])
        if yourdate.year in yearFreq:
            yearFreq[yourdate.year]+=1
        else:
            yearFreq[yourdate.year]=1
    return yearFreq
        

In [11]:
# Final_images

In [14]:
img_keys = list(Final_images.keys())

In [15]:
Final_images[img_keys[10]]

{'geometry': {'type': 'Point', 'coordinates': [-0.0152268, 51.4903976]},
 'type': 'Feature',
 'properties': {'username': 'gness',
  'sequence_key': '4W_dGaNuQW-xYFLqEOin8w',
  'camera_make': 'Apple',
  'user_key': 'FtLWg42ZUZzRDNaUUp-qAA',
  'ca': 270.98468017578125,
  'camera_model': 'iPhone',
  'key': '0euG8WgIdEH_k3DlgkLbkg',
  'pano': False,
  'captured_at': '2019-02-24T14:25:15.893Z'}}

In [16]:
Final_images[img_keys[10]]['geometry']['coordinates']

[-0.0152268, 51.4903976]

In [17]:
yearFreq = getYearDict(Final_images)

In [41]:
# yearFreq

# Uncomment below block if you want to download images

In [18]:
ImageDir = city + "_test_Mapillary_images"
for k in img_keys[:10]:
    ImageKey = Final_images[k]['properties']['key']
    url = mapillary_Image_url + ImageKey + "/thumb-640.jpg"
    logger.debug("Downloading: %s"%(url))
    response = rq.get(url)
    if response.status_code == 200:
        try:
            img = Image.open(BytesIO(response.content))
            filename = '%s/%s.%s' % (ImageDir, ImageKey, 'jpg')
            img.save(filename)
        except:
            logger.debug("Something bad happened while downloading: %s"%(url))
    else:
        logger.debug("Download failed: %s"%(url))

In [80]:
pandas_dict = {'Imgkey':[] , 'long': [] , 'lat': [], 'isPano':[] , 'captureTime':[], 'long_lat_string' : [] }  
for k in Final_images:
    pandas_dict['Imgkey'].append(Final_images[k]['properties']['key'])
    pandas_dict['isPano'].append(Final_images[k]['properties']['pano'])
    pandas_dict['captureTime'].append(Final_images[k]['properties']['captured_at'])
    pandas_dict['long'].append(Final_images[k]['geometry']['coordinates'][0])
    pandas_dict['lat'].append(Final_images[k]['geometry']['coordinates'][1])
    coords = Final_images[k]['geometry']['coordinates']
    pandas_dict['long_lat_string'].append(str(format(coords[0], '.4f')) + '_' + str(format(coords[1], '.4f')))
mapillary_df = pd.DataFrame.from_dict(pandas_dict)


In [81]:
len(mapillary_df['long_lat_string'].unique())

203367

In [82]:
mapillary_df.head()

,Imgkey,captureTime,isPano,lat,long,long_lat_string
0,SNkE9dXx2K0o3MaY9Zu-2A,2016-03-24T14:44:50.927Z,False,51.523651,-0.178409,-0.1784_51.5237
1,bneiFOYn9N2MQpjD-E7y-Q,2018-05-03T10:59:33.785Z,False,51.452568,-0.036538,-0.0365_51.4526
2,C5QMwGz01Rs6zk56q6fkeA,2018-12-09T15:42:45.172Z,False,51.462317,-0.389959,-0.3900_51.4623
3,YBQNM1wNdihc3tl1Jr1oDA,2014-08-04T10:19:53.000Z,False,51.508081,-0.095355,-0.0954_51.5081
4,6iT4CuOAHfSYNXWs_hlbdw,2015-11-25T13:50:11.000Z,False,51.485719,-0.120967,-0.1210_51.4857


In [83]:
mapillary_df.to_csv("Mappilary_points.csv")

In [84]:
test = pd.read_csv("Mappilary_points.csv")

In [85]:
test['long_lat_list'].unique()

KeyError: 'long_lat_list'